In [ ]:
import pip
pip.main(['install', 'pandas'])
pip.main(['install', 'matplotlib'])
pip.main(['install', 'seaborn'])
pip.main(['install', 'scikit-learn'])

CHARGEMENT DES BIBLIOTHEQUES

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


CHARGEMENT DU JEU DE DONNEES

In [5]:
dataset_path = "C:\\Users\\belga\\OneDrive\\Bureau\\EMBA Big Data\\Module Datascience\\Projet Data Science S2 2024\\CLIENT.csv"
CLIENT = pd.read_csv(dataset_path, sep='|', low_memory=False)
print(CLIENT.shape)
print(list(CLIENT.columns))

(845876, 10)
['IDCLIENT', 'CIVILITE', 'DATENAISSANCE', 'MAGASIN', 'DATEDEBUTADHESION', 'DATEREADHESION', 'DATEFINADHESION', 'VIP', 'CODEINSEE', 'PAYS']


La ligne de code ci-dessous nous permet de connaître le détail des colonnes ainsi que le type de données constituant notre base (numérique ou caractères).

In [6]:
CLIENT

,IDCLIENT,CIVILITE,DATENAISSANCE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,VIP,CODEINSEE,PAYS
0,155891.0,MADAME,NaN,VLG,13/05/2005 0:00:00,4/04/2017 0:00:00,30/04/2018 0:00:00,0,74224,FR
1,156023.0,MONSIEUR,7/08/1966 0:00:00,VLG,14/05/2005 0:00:00,21/06/2014 0:00:00,30/06/2016 0:00:00,0,74189,FR
2,124203.0,MADAME,NaN,BEA,16/03/2005 0:00:00,12/10/2017 0:00:00,31/10/2018 0:00:00,0,63070,FR
3,156190.0,MADAME,24/07/1976 0:00:00,SUR,28/08/2006 0:00:00,24/05/2015 0:00:00,31/05/2016 0:00:00,0,92026,FR
4,144444.0,MONSIEUR,NaN,LAB,29/04/2006 0:00:00,18/12/2016 0:00:00,31/12/2017 0:00:00,0,31555,FR
...,...,...,...,...,...,...,...,...,...,...
845871,1970111.0,MONSIEUR,NaN,IAB,14/04/2011 0:00:00,6/01/2018 0:00:00,31/01/2019 0:00:00,0,38053,FR
845872,2297275.0,MONSIEUR,6/02/1988 0:00:00,SEM,7/03/2013 0:00:00,6/01/2018 0:00:00,31/01/2019 0:00:00,0,57067,FR
845873,2305548.0,MONSIEUR,NaN,PRI,22/03/2013 0:00:00,6/01/2018 0:00:00,31/01/2019 0:00:00,0,38318,FR
845874,84175.0,MADAME,7/04/1950 0:00:00,GAI,29/10/2002 0:00:00,7/01/2018 0:00:00,31/01/2019 0:00:00,0,74082,FR


In [7]:
CLIENT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845876 entries, 0 to 845875
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   IDCLIENT           845876 non-null  float64
 1   CIVILITE           845876 non-null  object 
 2   DATENAISSANCE      507902 non-null  object 
 3   MAGASIN            845876 non-null  object 
 4   DATEDEBUTADHESION  845876 non-null  object 
 5   DATEREADHESION     507816 non-null  object 
 6   DATEFINADHESION    845876 non-null  object 
 7   VIP                845876 non-null  int64  
 8   CODEINSEE          818455 non-null  object 
 9   PAYS               845873 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 64.5+ MB


Nous allons procéder à la suppression des clients VIP, qui ont un comportement atypique qui viendra biaiser nos résultats.

In [8]:
CLIENT.head()

,IDCLIENT,CIVILITE,DATENAISSANCE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,VIP,CODEINSEE,PAYS
0,155891.0,MADAME,NaN,VLG,13/05/2005 0:00:00,4/04/2017 0:00:00,30/04/2018 0:00:00,0,74224,FR
1,156023.0,MONSIEUR,7/08/1966 0:00:00,VLG,14/05/2005 0:00:00,21/06/2014 0:00:00,30/06/2016 0:00:00,0,74189,FR
2,124203.0,MADAME,NaN,BEA,16/03/2005 0:00:00,12/10/2017 0:00:00,31/10/2018 0:00:00,0,63070,FR
3,156190.0,MADAME,24/07/1976 0:00:00,SUR,28/08/2006 0:00:00,24/05/2015 0:00:00,31/05/2016 0:00:00,0,92026,FR
4,144444.0,MONSIEUR,NaN,LAB,29/04/2006 0:00:00,18/12/2016 0:00:00,31/12/2017 0:00:00,0,31555,FR


Traitement des clients VIP, que nous ne souhaitons pas comptabiliser, car leur comportement atypique viendrait fausser nos résultats.

In [9]:
# Supprimer les lignes où la valeur de la colonne 'VIP' est égale à 1
CLIENT = CLIENT[CLIENT['VIP'] != 1]

# Vérifier la nouvelle forme du DataFrame
print(CLIENT.shape)

(735425, 10)


Suppression de la colonne VIP.

In [10]:
CLIENT = CLIENT.drop("VIP",axis=1)

Vérification du traitement.

In [11]:
CLIENT

,IDCLIENT,CIVILITE,DATENAISSANCE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,CODEINSEE,PAYS
0,155891.0,MADAME,NaN,VLG,13/05/2005 0:00:00,4/04/2017 0:00:00,30/04/2018 0:00:00,74224,FR
1,156023.0,MONSIEUR,7/08/1966 0:00:00,VLG,14/05/2005 0:00:00,21/06/2014 0:00:00,30/06/2016 0:00:00,74189,FR
2,124203.0,MADAME,NaN,BEA,16/03/2005 0:00:00,12/10/2017 0:00:00,31/10/2018 0:00:00,63070,FR
3,156190.0,MADAME,24/07/1976 0:00:00,SUR,28/08/2006 0:00:00,24/05/2015 0:00:00,31/05/2016 0:00:00,92026,FR
4,144444.0,MONSIEUR,NaN,LAB,29/04/2006 0:00:00,18/12/2016 0:00:00,31/12/2017 0:00:00,31555,FR
...,...,...,...,...,...,...,...,...,...
845871,1970111.0,MONSIEUR,NaN,IAB,14/04/2011 0:00:00,6/01/2018 0:00:00,31/01/2019 0:00:00,38053,FR
845872,2297275.0,MONSIEUR,6/02/1988 0:00:00,SEM,7/03/2013 0:00:00,6/01/2018 0:00:00,31/01/2019 0:00:00,57067,FR
845873,2305548.0,MONSIEUR,NaN,PRI,22/03/2013 0:00:00,6/01/2018 0:00:00,31/01/2019 0:00:00,38318,FR
845874,84175.0,MADAME,7/04/1950 0:00:00,GAI,29/10/2002 0:00:00,7/01/2018 0:00:00,31/01/2019 0:00:00,74082,FR


La ligne de code ci-dessous nous permet de formater le format de la date.

In [12]:
# Adapter le format de date pour les colonnes spécifiées
date_columns = ['DATENAISSANCE', 'DATEDEBUTADHESION', 'DATEREADHESION', 'DATEFINADHESION']
for col in date_columns:
    CLIENT[col] = pd.to_datetime(CLIENT[col], format='%d/%m/%Y %H:%M:%S',errors = 'coerce').dt.strftime('%d/%m/%Y')

Vérification du traitement.

In [13]:
CLIENT

,IDCLIENT,CIVILITE,DATENAISSANCE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,CODEINSEE,PAYS
0,155891.0,MADAME,NaN,VLG,13/05/2005,04/04/2017,30/04/2018,74224,FR
1,156023.0,MONSIEUR,07/08/1966,VLG,14/05/2005,21/06/2014,30/06/2016,74189,FR
2,124203.0,MADAME,NaN,BEA,16/03/2005,12/10/2017,31/10/2018,63070,FR
3,156190.0,MADAME,24/07/1976,SUR,28/08/2006,24/05/2015,31/05/2016,92026,FR
4,144444.0,MONSIEUR,NaN,LAB,29/04/2006,18/12/2016,31/12/2017,31555,FR
...,...,...,...,...,...,...,...,...,...
845871,1970111.0,MONSIEUR,NaN,IAB,14/04/2011,06/01/2018,31/01/2019,38053,FR
845872,2297275.0,MONSIEUR,06/02/1988,SEM,07/03/2013,06/01/2018,31/01/2019,57067,FR
845873,2305548.0,MONSIEUR,NaN,PRI,22/03/2013,06/01/2018,31/01/2019,38318,FR
845874,84175.0,MADAME,07/04/1950,GAI,29/10/2002,07/01/2018,31/01/2019,74082,FR


Cette ligne de code nous permet d'identifier le nombre d'occurence de chaque civilité, pour identifier les éventuels regroupements à effectuer. 

In [14]:
#modalités variables:
CLIENT['CIVILITE'].value_counts(dropna = False)

CIVILITE
MADAME      393703
MONSIEUR    337360
Mr            1431
Mme           1284
madame         976
monsieur       671
Name: count, dtype: int64

Il convient de procéder aux regroupements ci-dessous.

In [15]:
#regroupement des modalités:
CLIENT['CIVILITE']=np.where(CLIENT['CIVILITE'] =='MADAME', 'Madame', CLIENT['CIVILITE']) 
CLIENT['CIVILITE']=np.where(CLIENT['CIVILITE'] =='Mme', 'Madame', CLIENT['CIVILITE'])
CLIENT['CIVILITE']=np.where(CLIENT['CIVILITE'] =='madame', 'Madame', CLIENT['CIVILITE']) 
CLIENT['CIVILITE']=np.where(CLIENT['CIVILITE'] =='MONSIEUR', 'Monsieur', CLIENT['CIVILITE'])
CLIENT['CIVILITE']=np.where(CLIENT['CIVILITE'] =='Mr', 'Monsieur', CLIENT['CIVILITE'])
CLIENT['CIVILITE']=np.where(CLIENT['CIVILITE'] =='monsieur', 'Monsieur', CLIENT['CIVILITE'])

Vérification du traitement.

In [16]:
CLIENT['CIVILITE'].value_counts(dropna = False)

CIVILITE
Madame      395963
Monsieur    339462
Name: count, dtype: int64

Traitement de la civilité en variable "sexe" pour plus de pertinence.

In [17]:
# Mapping de la colonne CIVILITE pour créer la nouvelle colonne SEXE
CLIENT['SEXE'] = CLIENT['CIVILITE'].map({'Monsieur': 'HOMME', 'Madame': 'FEMME'})

print(CLIENT)


         IDCLIENT  CIVILITE DATENAISSANCE MAGASIN DATEDEBUTADHESION  \
0        155891.0    Madame           NaN     VLG        13/05/2005   
1        156023.0  Monsieur    07/08/1966     VLG        14/05/2005   
2        124203.0    Madame           NaN     BEA        16/03/2005   
3        156190.0    Madame    24/07/1976     SUR        28/08/2006   
4        144444.0  Monsieur           NaN     LAB        29/04/2006   
...           ...       ...           ...     ...               ...   
845871  1970111.0  Monsieur           NaN     IAB        14/04/2011   
845872  2297275.0  Monsieur    06/02/1988     SEM        07/03/2013   
845873  2305548.0  Monsieur           NaN     PRI        22/03/2013   
845874    84175.0    Madame    07/04/1950     GAI        29/10/2002   
845875  1280004.0  Monsieur           NaN     SGL        23/06/2007   

       DATEREADHESION DATEFINADHESION CODEINSEE PAYS   SEXE  
0          04/04/2017      30/04/2018     74224   FR  FEMME  
1          21/06/2014  

In [18]:
#Grouper les Clients par Sexe
clients_par_sexe = CLIENT.groupby('SEXE')['IDCLIENT'].nunique().reset_index(name='NbClients')

print(clients_par_sexe)

    SEXE  NbClients
0  FEMME     395963
1  HOMME     339462


Suppression de la colonne "civilité" qui n'a plus lieu d'être.

In [19]:
# Supprimer l'ancienne colonne CIVILITE
CLIENT.drop(columns=['CIVILITE'], inplace=True)

Affichage de notre dataframe.

In [20]:
CLIENT

,IDCLIENT,DATENAISSANCE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,CODEINSEE,PAYS,SEXE
0,155891.0,NaN,VLG,13/05/2005,04/04/2017,30/04/2018,74224,FR,FEMME
1,156023.0,07/08/1966,VLG,14/05/2005,21/06/2014,30/06/2016,74189,FR,HOMME
2,124203.0,NaN,BEA,16/03/2005,12/10/2017,31/10/2018,63070,FR,FEMME
3,156190.0,24/07/1976,SUR,28/08/2006,24/05/2015,31/05/2016,92026,FR,FEMME
4,144444.0,NaN,LAB,29/04/2006,18/12/2016,31/12/2017,31555,FR,HOMME
...,...,...,...,...,...,...,...,...,...
845871,1970111.0,NaN,IAB,14/04/2011,06/01/2018,31/01/2019,38053,FR,HOMME
845872,2297275.0,06/02/1988,SEM,07/03/2013,06/01/2018,31/01/2019,57067,FR,HOMME
845873,2305548.0,NaN,PRI,22/03/2013,06/01/2018,31/01/2019,38318,FR,HOMME
845874,84175.0,07/04/1950,GAI,29/10/2002,07/01/2018,31/01/2019,74082,FR,FEMME


Cette ligne de code nous permet d'identifier le nombre d'occurence de chaque magasin, pour identifier les éventuels regroupements à effectuer.

In [21]:
CLIENT['MAGASIN'].value_counts(dropna = False)

MAGASIN
PRI    23963
SEY    23572
VIB    22223
SUR    20923
MOU    20887
       ...  
VIV     3187
SAL     2836
STR     2667
EST     1145
BAR     1039
Name: count, Length: 67, dtype: int64

Les données à traiter concernant la France, la série de code ci-dessous nous permet de ne conserver que ce pays.

In [22]:
# Filtrer les lignes pour ne garder que celles avec "France" dans la colonne "Pays"
CLIENT = CLIENT[CLIENT['PAYS'] == 'FR']

# Réindexer le DataFrame après avoir filtré les lignes
CLIENT.reset_index(drop=True, inplace=True)

In [23]:
CLIENT['PAYS'].value_counts(dropna = False)

PAYS
FR    720090
Name: count, dtype: int64

La serie de codes ci-dessous nous permet de traiter la date de naissance des clients afin d'en déterminer l'âge puis des tranches d'âges.

In [ ]:
#Convertir la colonne de date de naissance en objet datetime :

CLIENT['DATENAISSANCE'] = pd.to_datetime(CLIENT['DATENAISSANCE'],errors = 'coerce')

#En créer une nouvelle colonne pour l'âge en calculant la différence entre la date actuelle et la date de naissance :

from datetime import datetime
today = datetime.today()
CLIENT['AGE'] = today.year - CLIENT['DATENAISSANCE'].dt.year
#Discrétiser l'âge en créant des catégories d'âge. Par exemple, diviser en groupes d'âge :
bins = [18, 30, 45, 60, 100]
CLIENT['AGE_GROUP'] = pd.cut(CLIENT['AGE'], bins, labels=['AGE DE 18-30', 'AGE DE 31-45', 'AGE DE 46-60', 'AGE DE 61-100'])

Vérification du traitement.

In [25]:
CLIENT['AGE_GROUP'].value_counts(dropna = False)

AGE_GROUP
NaN              548518
AGE DE 61-100     67546
AGE DE 46-60      57057
AGE DE 31-45      42700
AGE DE 18-30       4269
Name: count, dtype: int64

Après ce traitement, il convient de renommer la variable concernée.

In [26]:
# Renommer la colonne 'DATENAISSANCE' en 'AGE'
CLIENT = CLIENT.rename(columns={'DATENAISSANCE': 'AGE'})

Vérification du traitement.

In [27]:
CLIENT

,IDCLIENT,AGE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,CODEINSEE,PAYS,SEXE,AGE,AGE_GROUP
0,155891.0,NaT,VLG,13/05/2005,04/04/2017,30/04/2018,74224,FR,FEMME,NaN,NaN
1,156023.0,1966-07-08,VLG,14/05/2005,21/06/2014,30/06/2016,74189,FR,HOMME,58.0,AGE DE 46-60
2,124203.0,NaT,BEA,16/03/2005,12/10/2017,31/10/2018,63070,FR,FEMME,NaN,NaN
3,156190.0,NaT,SUR,28/08/2006,24/05/2015,31/05/2016,92026,FR,FEMME,NaN,NaN
4,144444.0,NaT,LAB,29/04/2006,18/12/2016,31/12/2017,31555,FR,HOMME,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
720085,1970111.0,NaT,IAB,14/04/2011,06/01/2018,31/01/2019,38053,FR,HOMME,NaN,NaN
720086,2297275.0,1988-06-02,SEM,07/03/2013,06/01/2018,31/01/2019,57067,FR,HOMME,36.0,AGE DE 31-45
720087,2305548.0,NaT,PRI,22/03/2013,06/01/2018,31/01/2019,38318,FR,HOMME,NaN,NaN
720088,84175.0,1950-07-04,GAI,29/10/2002,07/01/2018,31/01/2019,74082,FR,FEMME,74.0,AGE DE 61-100


Afin d'exploiter de façon optimale les informations, nous allons convetir les l'ID TICKET, l'âge et les tranches d'âges en caractère. Nous n'avons pas nécessité à conserver ces valeurs en numérique.

In [28]:
CLIENT['IDCLIENT'] = CLIENT['IDCLIENT'].astype(str)
CLIENT['AGE'] = CLIENT['AGE'].astype(str)
CLIENT['AGE_GROUP'] = CLIENT['AGE_GROUP'].astype(str)

Vérification du traitement.

In [29]:
CLIENT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720090 entries, 0 to 720089
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   IDCLIENT           720090 non-null  object
 1   AGE                720090 non-null  object
 2   MAGASIN            720090 non-null  object
 3   DATEDEBUTADHESION  720090 non-null  object
 4   DATEREADHESION     410990 non-null  object
 5   DATEFINADHESION    720090 non-null  object
 6   CODEINSEE          709392 non-null  object
 7   PAYS               720090 non-null  object
 8   SEXE               720090 non-null  object
 9   AGE                720090 non-null  object
 10  AGE_GROUP          720090 non-null  object
dtypes: object(11)
memory usage: 60.4+ MB


La ligne de code ci-dessous nous permet d'identifier les valeurs en doublons qui conviendra de traiter.

In [30]:
print(CLIENT.duplicated().sum())
CLIENT = CLIENT.drop_duplicates()

0


Nous procédons également à la vérification de présence ou non de valeurs manquantes.

In [31]:
pd.DataFrame(index=CLIENT.columns, columns=['%_missing_values'], data= (CLIENT.isna().sum().values / len(CLIENT)*100))

,%_missing_values
IDCLIENT,0.000000
AGE,0.000000
MAGASIN,0.000000
DATEDEBUTADHESION,0.000000
DATEREADHESION,42.925190
DATEFINADHESION,0.000000
CODEINSEE,1.485648
PAYS,0.000000
SEXE,0.000000
AGE,0.000000


La série de codes ci-dessous nous permet de traiter les valeurs manquantes détéctées, sur les variables concernées.

In [32]:
CLIENT['CODEINSEE'].isnull()

0         False
1         False
2         False
3         False
4         False
          ...  
720085    False
720086    False
720087    False
720088    False
720089    False
Name: CODEINSEE, Length: 720090, dtype: bool

In [33]:
CLIENT.dropna(subset=['CODEINSEE'], inplace=True)

In [34]:
CLIENT['DATEREADHESION'].isnull()

0         False
1         False
2         False
3         False
4         False
          ...  
720085    False
720086    False
720087    False
720088    False
720089    False
Name: DATEREADHESION, Length: 709392, dtype: bool

In [35]:
CLIENT.dropna(subset=['DATEREADHESION'], inplace=True)

In [36]:
CLIENT['AGE_GROUP'].isnull()

0         False
1         False
2         False
3         False
4         False
          ...  
720085    False
720086    False
720087    False
720088    False
720089    False
Name: AGE_GROUP, Length: 407389, dtype: bool

In [37]:
CLIENT['AGE_GROUP'].value_counts()

AGE_GROUP
nan              326057
AGE DE 61-100     39498
AGE DE 46-60      27990
AGE DE 31-45      13240
AGE DE 18-30        604
Name: count, dtype: int64

In [38]:
CLIENT['AGE_GROUP'] = CLIENT['AGE_GROUP'].replace(np.nan, "Inconnue")


Remplacement des valeurs manquantes par une modalité 'inconnue".

In [39]:
# Utilisation de `map` pour remplacer 'nan' par 'Inconnue'
CLIENT['AGE_GROUP'] = CLIENT['AGE_GROUP'].map(lambda x: 'Inconnue' if x == 'nan' else x)

# Affichage du résultat
print(CLIENT)

         IDCLIENT         AGE MAGASIN DATEDEBUTADHESION DATEREADHESION  \
0        155891.0         NaT     VLG        13/05/2005     04/04/2017   
1        156023.0  1966-07-08     VLG        14/05/2005     21/06/2014   
2        124203.0         NaT     BEA        16/03/2005     12/10/2017   
3        156190.0         NaT     SUR        28/08/2006     24/05/2015   
4        144444.0         NaT     LAB        29/04/2006     18/12/2016   
...           ...         ...     ...               ...            ...   
720085  1970111.0         NaT     IAB        14/04/2011     06/01/2018   
720086  2297275.0  1988-06-02     SEM        07/03/2013     06/01/2018   
720087  2305548.0         NaT     PRI        22/03/2013     06/01/2018   
720088    84175.0  1950-07-04     GAI        29/10/2002     07/01/2018   
720089  1280004.0         NaT     SGL        23/06/2007     07/01/2018   

       DATEFINADHESION CODEINSEE PAYS   SEXE   AGE      AGE_GROUP  
0           30/04/2018     74224   FR  FEMM

In [40]:
CLIENT['AGE_GROUP'].value_counts()

AGE_GROUP
Inconnue         326057
AGE DE 61-100     39498
AGE DE 46-60      27990
AGE DE 31-45      13240
AGE DE 18-30        604
Name: count, dtype: int64

In [41]:
CLIENT.dropna(subset=['AGE_GROUP'], inplace=True)

Vérification du taitement.

In [42]:
CLIENT

,IDCLIENT,AGE,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,CODEINSEE,PAYS,SEXE,AGE,AGE_GROUP
0,155891.0,NaT,VLG,13/05/2005,04/04/2017,30/04/2018,74224,FR,FEMME,nan,Inconnue
1,156023.0,1966-07-08,VLG,14/05/2005,21/06/2014,30/06/2016,74189,FR,HOMME,58.0,AGE DE 46-60
2,124203.0,NaT,BEA,16/03/2005,12/10/2017,31/10/2018,63070,FR,FEMME,nan,Inconnue
3,156190.0,NaT,SUR,28/08/2006,24/05/2015,31/05/2016,92026,FR,FEMME,nan,Inconnue
4,144444.0,NaT,LAB,29/04/2006,18/12/2016,31/12/2017,31555,FR,HOMME,nan,Inconnue
...,...,...,...,...,...,...,...,...,...,...,...
720085,1970111.0,NaT,IAB,14/04/2011,06/01/2018,31/01/2019,38053,FR,HOMME,nan,Inconnue
720086,2297275.0,1988-06-02,SEM,07/03/2013,06/01/2018,31/01/2019,57067,FR,HOMME,36.0,AGE DE 31-45
720087,2305548.0,NaT,PRI,22/03/2013,06/01/2018,31/01/2019,38318,FR,HOMME,nan,Inconnue
720088,84175.0,1950-07-04,GAI,29/10/2002,07/01/2018,31/01/2019,74082,FR,FEMME,74.0,AGE DE 61-100


La ligne de code ci-dessous nous permet d'identifier la présence ou non de valeur(s) manquante(s)

In [43]:
pd.DataFrame(index=CLIENT.columns, columns=['%_missing_values'], data= (CLIENT.isna().sum().values / len(CLIENT)*100))

,%_missing_values
IDCLIENT,0.0
AGE,0.0
MAGASIN,0.0
DATEDEBUTADHESION,0.0
DATEREADHESION,0.0
DATEFINADHESION,0.0
CODEINSEE,0.0
PAYS,0.0
SEXE,0.0
AGE,0.0


Le traitement des valeurs aberrantes ne concernant que les variables numériques, nous n'allons pas procéder à ce traitement car aucune de nos variables n'est concernées sur ce dataframe.

Suppression des colonnes dont les informations nous serons pas nécessairement utiles dans le cadre de notre projet.

In [44]:
CLIENT.drop(columns=['PAYS'], inplace=True)

In [45]:
CLIENT.drop(columns=['AGE'], inplace=True)

Affichage de notre dataframe.

In [46]:
CLIENT

,IDCLIENT,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,CODEINSEE,SEXE,AGE_GROUP
0,155891.0,VLG,13/05/2005,04/04/2017,30/04/2018,74224,FEMME,Inconnue
1,156023.0,VLG,14/05/2005,21/06/2014,30/06/2016,74189,HOMME,AGE DE 46-60
2,124203.0,BEA,16/03/2005,12/10/2017,31/10/2018,63070,FEMME,Inconnue
3,156190.0,SUR,28/08/2006,24/05/2015,31/05/2016,92026,FEMME,Inconnue
4,144444.0,LAB,29/04/2006,18/12/2016,31/12/2017,31555,HOMME,Inconnue
...,...,...,...,...,...,...,...,...
720085,1970111.0,IAB,14/04/2011,06/01/2018,31/01/2019,38053,HOMME,Inconnue
720086,2297275.0,SEM,07/03/2013,06/01/2018,31/01/2019,57067,HOMME,AGE DE 31-45
720087,2305548.0,PRI,22/03/2013,06/01/2018,31/01/2019,38318,HOMME,Inconnue
720088,84175.0,GAI,29/10/2002,07/01/2018,31/01/2019,74082,FEMME,AGE DE 61-100


La série de codes ci-dessous nous permet de modifier le format des dates de début et de fin d'adhésion afin de calculer l'ancienneté des clients.

In [47]:
import datetime
from datetime import timedelta

In [48]:
# Correction de l'Avertissement en spécifiant dayfirst=True
CLIENT['DATEDEBUTADHESION'] = pd.to_datetime(CLIENT['DATEDEBUTADHESION'], dayfirst=True)

In [49]:
CLIENT['DATEDEBUTADHESION'] = CLIENT['DATEDEBUTADHESION'].dt.year.astype(int)

In [50]:
CLIENT['DATEFINADHESION'] = pd.to_datetime(CLIENT['DATEFINADHESION'],dayfirst=True)

In [51]:
CLIENT['DATEFINADHESION'] = CLIENT['DATEFINADHESION'].dt.year.astype(int)

In [52]:
if 'DATEDEBUTADHESION' in CLIENT.columns:
    Date_début = CLIENT['DATEDEBUTADHESION']
    print(Date_début.head())
else:
    print("La colonne 'DATEDEBUTADHESION' n'existe pas dans le DataFrame.")

0    2005
1    2005
2    2005
3    2006
4    2006
Name: DATEDEBUTADHESION, dtype: int32


In [53]:
if 'DATEFINADHESION' in CLIENT.columns:
    Date_fin = CLIENT['DATEFINADHESION']
    print(Date_fin.head())
else:
    print("La colonne 'DATEFINADHESION' n'existe pas dans le DataFrame.")

0    2018
1    2016
2    2018
3    2016
4    2017
Name: DATEFINADHESION, dtype: int32


In [54]:
ANCIENNETE = Date_fin - Date_début

In [55]:
print(ANCIENNETE)

0         13
1         11
2         13
3         10
4         11
          ..
720085     8
720086     6
720087     6
720088    17
720089    12
Length: 407389, dtype: int32


Nous allons maintenant intégrer cette variables d'ancienneté à notre jeu de données.

In [56]:
CLIENT.insert(loc=6, column='ANCIENNETE', value=ANCIENNETE)

Affichage de notre dataframe.

In [57]:
CLIENT

,IDCLIENT,MAGASIN,DATEDEBUTADHESION,DATEREADHESION,DATEFINADHESION,CODEINSEE,ANCIENNETE,SEXE,AGE_GROUP
0,155891.0,VLG,2005,04/04/2017,2018,74224,13,FEMME,Inconnue
1,156023.0,VLG,2005,21/06/2014,2016,74189,11,HOMME,AGE DE 46-60
2,124203.0,BEA,2005,12/10/2017,2018,63070,13,FEMME,Inconnue
3,156190.0,SUR,2006,24/05/2015,2016,92026,10,FEMME,Inconnue
4,144444.0,LAB,2006,18/12/2016,2017,31555,11,HOMME,Inconnue
...,...,...,...,...,...,...,...,...,...
720085,1970111.0,IAB,2011,06/01/2018,2019,38053,8,HOMME,Inconnue
720086,2297275.0,SEM,2013,06/01/2018,2019,57067,6,HOMME,AGE DE 31-45
720087,2305548.0,PRI,2013,06/01/2018,2019,38318,6,HOMME,Inconnue
720088,84175.0,GAI,2002,07/01/2018,2019,74082,17,FEMME,AGE DE 61-100


Ci-dessous, nous procédons à la suppression des dates de début et de fin d'adhésion car nous n'en avons plus besoin.

In [58]:
CLIENT.drop(columns=['DATEDEBUTADHESION'], inplace=True)

In [59]:
CLIENT.drop(columns=['DATEFINADHESION'], inplace=True)

Vérification des traitements.

In [60]:
CLIENT

,IDCLIENT,MAGASIN,DATEREADHESION,CODEINSEE,ANCIENNETE,SEXE,AGE_GROUP
0,155891.0,VLG,04/04/2017,74224,13,FEMME,Inconnue
1,156023.0,VLG,21/06/2014,74189,11,HOMME,AGE DE 46-60
2,124203.0,BEA,12/10/2017,63070,13,FEMME,Inconnue
3,156190.0,SUR,24/05/2015,92026,10,FEMME,Inconnue
4,144444.0,LAB,18/12/2016,31555,11,HOMME,Inconnue
...,...,...,...,...,...,...,...
720085,1970111.0,IAB,06/01/2018,38053,8,HOMME,Inconnue
720086,2297275.0,SEM,06/01/2018,57067,6,HOMME,AGE DE 31-45
720087,2305548.0,PRI,06/01/2018,38318,6,HOMME,Inconnue
720088,84175.0,GAI,07/01/2018,74082,17,FEMME,AGE DE 61-100


In [61]:
CLIENT.info()

<class 'pandas.core.frame.DataFrame'>
Index: 407389 entries, 0 to 720089
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   IDCLIENT        407389 non-null  object
 1   MAGASIN         407389 non-null  object
 2   DATEREADHESION  407389 non-null  object
 3   CODEINSEE       407389 non-null  object
 4   ANCIENNETE      407389 non-null  int32 
 5   SEXE            407389 non-null  object
 6   AGE_GROUP       407389 non-null  object
dtypes: int32(1), object(6)
memory usage: 23.3+ MB


Le code ci-dessous va également nous permettre de modifier le format de la date de ré-adhésion.

In [62]:
CLIENT["DATEREADHESION"] = pd.to_datetime(CLIENT['DATEREADHESION'].str.strip(), format='%d/%m/%Y')

In [63]:
# Convertir la colonne 'DATEREADHESION' au format datetime
CLIENT['DATEREADHESION'] = pd.to_datetime(CLIENT['DATEREADHESION'], dayfirst=True)

# Afficher le DataFrame initial
print("Avant le filtrage:")
print(CLIENT)

# Définir la date limite pour le filtrage
date_max = pd.to_datetime('01/01/2018', dayfirst=True)

# Filtrer le DataFrame selon la condition
CLIENT = CLIENT[CLIENT['DATEREADHESION'] < date_max]

# Afficher le DataFrame après filtrage
print("\nAprès le filtrage:")
print(CLIENT)

Avant le filtrage:
         IDCLIENT MAGASIN DATEREADHESION CODEINSEE  ANCIENNETE   SEXE  \
0        155891.0     VLG     2017-04-04     74224          13  FEMME   
1        156023.0     VLG     2014-06-21     74189          11  HOMME   
2        124203.0     BEA     2017-10-12     63070          13  FEMME   
3        156190.0     SUR     2015-05-24     92026          10  FEMME   
4        144444.0     LAB     2016-12-18     31555          11  HOMME   
...           ...     ...            ...       ...         ...    ...   
720085  1970111.0     IAB     2018-01-06     38053           8  HOMME   
720086  2297275.0     SEM     2018-01-06     57067           6  HOMME   
720087  2305548.0     PRI     2018-01-06     38318           6  HOMME   
720088    84175.0     GAI     2018-01-07     74082          17  FEMME   
720089  1280004.0     SGL     2018-01-07     42218          12  HOMME   

            AGE_GROUP  
0            Inconnue  
1        AGE DE 46-60  
2            Inconnue  
3       

Affichage de notre dataframe.

In [64]:
CLIENT

,IDCLIENT,MAGASIN,DATEREADHESION,CODEINSEE,ANCIENNETE,SEXE,AGE_GROUP
0,155891.0,VLG,2017-04-04,74224,13,FEMME,Inconnue
1,156023.0,VLG,2014-06-21,74189,11,HOMME,AGE DE 46-60
2,124203.0,BEA,2017-10-12,63070,13,FEMME,Inconnue
3,156190.0,SUR,2015-05-24,92026,10,FEMME,Inconnue
4,144444.0,LAB,2016-12-18,31555,11,HOMME,Inconnue
...,...,...,...,...,...,...,...
719975,336771.0,BEA,2017-12-30,63302,12,FEMME,Inconnue
719976,1709516.0,VIB,2017-12-30,69034,9,HOMME,Inconnue
719977,1866695.0,MAN,2017-12-30,04230,8,FEMME,Inconnue
719978,2098666.0,CLI,2017-12-30,08105,6,HOMME,Inconnue


Enregistrement de notre dataframe pour faciliter sa réutilisation, dans un autre notebook", sans avoir à tout éxécuter.

In [65]:
CLIENTS = "C:\\Users\\belga\\OneDrive\\Bureau\\EMBA Big Data\\Module Datascience\\Projet Data Science S2 2024\\CLIENTS_NET.pkl"
CLIENT.to_pickle(CLIENTS)